# Imports & Paths

In [ ]:

import pandas as pd
from pathlib import Path

RAW_PATH = Path("../data/raw/tabela-fipe-historico-precos.csv")
PROC_DIR = Path("../data/processed")
PROC_DIR.mkdir(parents=True, exist_ok=True)
OUT_PATH = PROC_DIR / "fipe_processed.parquet"

pd.set_option("display.max_columns", None)


# Carga de Dados


In [2]:
# %%
df = pd.read_csv(RAW_PATH, sep=",", encoding="utf-8", low_memory=False)
print(df.shape)
df.head()

(466020, 8)


,Unnamed: 0,codigoFipe,marca,modelo,anoModelo,mesReferencia,anoReferencia,valor
0,0,038003-2,Acura,Integra GS 1.8,1992,11,2016,13041.0
1,1,038001-6,Acura,NSX 3.0,1995,3,2013,52339.0
2,2,038003-2,Acura,Integra GS 1.8,1992,3,2018,12423.0
3,3,038002-4,Acura,Legend 3.2/3.5,1998,5,2016,31067.0
4,4,038002-4,Acura,Legend 3.2/3.5,1998,2,2019,26381.0


# Seleção de Colunas e PAdronização de Tipos

In [3]:
keep = ["marca","modelo","anoModelo","mesReferencia","anoReferencia","valor"]
df = df[keep].copy()

# Tipagem
df["anoModelo"] = pd.to_numeric(df["anoModelo"], errors="coerce").astype("Int64")
df["mesReferencia"] = pd.to_numeric(df["mesReferencia"], errors="coerce").astype("Int64")
df["anoReferencia"] = pd.to_numeric(df["anoReferencia"], errors="coerce").astype("Int64")
df["valor"] = pd.to_numeric(df["valor"], errors="coerce")

# Strings
df["marca"] = df["marca"].astype(str).str.strip()
df["modelo"] = df["modelo"].astype(str).str.strip()

df.dtypes


marca             object
modelo            object
anoModelo          Int64
mesReferencia      Int64
anoReferencia      Int64
valor            float64
dtype: object

# Remoção de Nulos

In [4]:

before = df.shape[0]
df = df.dropna(subset=["marca","modelo","anoModelo","mesReferencia","anoReferencia","valor"])
after = df.shape[0]
print(f"Removidos por nulos essenciais: {before - after}")


Removidos por nulos essenciais: 0


# Criar Data Referencia

In [5]:
df["dataReferencia"] = pd.to_datetime(
    dict(year=df["anoReferencia"].astype(int), 
         month=df["mesReferencia"].astype(int), 
         day=1),
    errors="coerce"
)
null_dates = df["dataReferencia"].isna().sum()
print("datas inválidas:", null_dates)

# Remover registros com data inválida (se houver)
df = df.dropna(subset=["dataReferencia"])


datas inválidas: 0


# Tratamento de Outliers

In [ ]:

MIN_VALOR = 500.0
MAX_VALOR = 2_000_000.0

mask = (df["valor"] >= MIN_VALOR) & (df["valor"] <= MAX_VALOR)
removed = (~mask).sum()
df = df.loc[mask].copy()

print(f"Removidos por faixa de valor ({MIN_VALOR:.0f}–{MAX_VALOR:.0f}): {removed}")


# Remoção de Duplicados

In [6]:
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"Removidos duplicados exatos: {before - after}")


Removidos duplicados exatos: 0


# Checagem pós tratamento

In [7]:

print("Formato final:", df.shape)
print("\nAmostra:")
display(df.sample(5, random_state=42))

print("\nResumo de 'valor':")
display(df["valor"].describe(percentiles=[0.05,0.25,0.5,0.75,0.95]))


Formato final: (466020, 7)

Amostra:


,marca,modelo,anoModelo,mesReferencia,anoReferencia,valor,dataReferencia
162623,Jaguar,F-Pace 3.0 S 380cv Aut.,2022,8,2019,430416.0,2019-08-01
11200,Audi,Q3 1.4 TFSI/TFSI Flex S-tronic 5p,2022,8,2016,149393.0,2016-08-01
366001,Saab,9000 CD 2.3 Turbo,1993,2,2011,18060.0,2011-02-01
226775,Mitsubishi,Space Wagon GLS 2.0,1995,7,2011,13090.0,2011-07-01
421637,Volvo,C70 Cabriolet,2022,2,2013,189660.0,2013-02-01



Resumo de 'valor':


count    4.660200e+05
mean     1.157456e+05
std      3.250737e+05
min      1.871000e+03
5%       8.333000e+03
25%      1.938800e+04
50%      3.941650e+04
75%      8.479450e+04
95%      4.400000e+05
max      8.407082e+06
Name: valor, dtype: float64

# Salvar Processado

In [11]:
df.to_parquet(OUT_PATH, index=False, engine="fastparquet")
OUT_PATH

PosixPath('../data/processed/fipe_processed.parquet')